O código abaixo foi usado como exemplo de projeto no Bootcamp da DIO para cartificação AI-190. Faz-se necessário um endpoint e chave válidos para que o código funcione.

O exercício feito no VS Code em classes separadas está escrito aqui no colab.

In [9]:
pip install azure.core

In [11]:
pip install azure-ai-documentintelligence

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 kB 4.5 MB/s eta 0:00:00


In [13]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.8 MB/s eta 0:00:00


In [14]:
pip install azure-storage-blob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.9/428.9 kB 7.5 MB/s eta 0:00:00


In [15]:
pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

In [5]:
#Classe Config
load_dotenv()

class Config:
  ENDPOINT = os.getenv("ENDPOINT")
  KEY = os.getenv("SUBSCRIPTION_KEY")
  AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
  CONTAINER_NAME = os.getenv("CONTAINER_NAME")

#Depois de fazer upload do arquivo, é preciso colocar o arquivo no storage pegar
#a URL e mandar para o Document Inteligence e conferir se é um arquivo válido.

In [16]:
#app.py

import streamlit as st
from services.blob_servide import upload_blob
from sercives.credit_card_services import analize_credit_card

def configure_interface():
  st.title("Upload de arquivos DIO - Disafio 2 - Azure - Fake Docs")
  upload_file = st.file_uploader("Escolha um arquivo", type = ["png", "jpg", "jpeg"])

if uploaded_file is not None:
  fileName = uploaded_file.name
  #Enviar para o blob storage
  blob_url = upload_blob(uploaded_file, fileName)
  if blob_url: #is not None
    st.write(f"Arquivo {file_name} enviado com sucesso para o Azure Blob Storage")
    #Uma vez que o arquivo foi, preciso validar se é um cartão de crédito
    credit_card_info = analize_credit_card (blob_url) #Chamar a função de detecção de informações de cartão de crédito
    show_image_and_validation(blob_url, credit_card_info)
  else:
    st.write(f"Erro ao enviar o arquivo {file_name} para o Azure Blob Storage")

def show_image_and_validation(blob_url, credit_card_info):
  st.image(blob_url, caption = "Imagem enviada", use_columns_width = True)
  st.write("Informações de cartão de crédito encontradas: ")
  st.write(credit_card_info)
  #Validações
  if credit_card_info and credit_card_info["card_name"]:
    st.markdown(f"<h1 style = 'color:green; >Cartão Válido</h1>", unsafe_allow_html = True)
    st.write(f"Nome do titular = {credit_card_info["card_name"]}")
    st.write(f"Banco emissor = {credit_card_info["bank_name"]}")
    st.write(f"Data de validade = {credit_card_info["expiry_date"]}")
  else:
    st.markdown(f"<h1 style = 'color:red; >Cartão álido</h1>", unsafe_allow_html = True)
    st.write("Esse não é um número de cartão válido")


if __name__ == "__main__":
  configure_interface()

NameError: name 'uploaded_file' is not defined

In [ ]:
#Classe de Blob
import os
from azure.storage.blob import BlobServiceClient
import streamlit as st
from ultils.Config import Config

def upload_blob(file, file_name):
  try:
    blob_service_client = BlobServiceClient.from_connection_string(Config.AZURE_STORAGE_CONNECTION_STRING)
    blob_client = blob_service_client.get_blob_client(container = Config.CONTAINER_NAME, blob=file_name)
    blob.client.upload_blob(file, overwrite = True)
  return blob_client.url
except Exception as ex:
  st.error(f"Erro ao enviar o arquivo para o Azure Blob Storage: {ex}")
  return None

In [ ]:
#credit_card_service.py
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentAnalysisClient
from azure.ai.documentintelligence.models import AnaluseResult
from ultils.Config import Config

def analize_credit_card(card_url):
  credential = AzureKeyCredential(Config.KEY)
  document_inteligence_client = DocumentIntelligenceClient(Config.ENDPOINT, credential)
  card_info = document_Client.begin_analyze_document(
        "prebuild-creditCard", AnalyzeDocumentRequest(url_source_url))
  result = card_info.result()

  for document in result.documents:
    fields = document.get('fields',{})

  #Para retornar um result mais organizado
  return{
     "card_name": fields.get('CardHolderName', {}).get('content'),
     "card_number": fields.get('CardNumber', {}).get('content'),
     "expiry_date": fields.get('ExpirationDate', {}).get('content'),
     "bank_name": fields.get('IssuingBank', {}).get('content'),
     }